This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# stored API key in a file called 'apikey',
# not provided in GitHub
filename = 'apikey'

In [2]:
# define function to parse API Key from file
# and return it to be stored as string
def get_file_contents(filename):
    """ Given a filename, return the contents of that file"""
    try:
        with open(filename, 'r') as f:
            # specific file contains a single line, with API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [3]:
# store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = get_file_contents(filename)

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# first, import the relevant modules
#import collections
import requests
import json

In [5]:
# now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r1 = requests.get("https://API_KEY@www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2018-08-23&end_date=2018-08-23")

In [6]:
# inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r1.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2018-08-24T22:28:36.883Z',
  'newest_available_date': '2018-08-24',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2018-08-23',
  'end_date': '2018-08-23',
  'data': [['2018-08-23',
    None,
    75.55,
    73.85,
    74.6,
    None,
    77293.0,
    5775269.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
***

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017.

In [7]:
# use .get(url) to request data from Quandl repository and store as Response
# provide API_KEY to access data
# specify {database_code} = FSE and {dataset_code} = AFX_X
# provide start/end dates to get data for 2017
r1 = requests.get("https://API_KEY@www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31")

***
### 2. Convert the returned JSON object into a Python dictionary.

In [8]:
# use .json() method to convert Response to Python dictionary
# display data type
json_dict = r1.json()
type(json_dict)

dict

***
#### For the remaining questions in this project, I only plan to use the values associated with the `'data'` key from the time-series data set.

Example:
```python 
'data': [['2018-08-23', None, 75.55, 73.85, 74.6, None, 77293.0, 5775269.0, None, None, None]]
```
The values correspond to the following column names:
```python
['Date','Open','High','Low','Close','Change','Traded Volume','Turnover',
 'Last Price of the Day','Daily Traded Units','Daily Turnover']
```
*Some values contain* **`None`**'*s and will be addressed by excluding them from calculations.*

In [9]:
# parse only the values for 'data' from dictionary and store in list
vals = json_dict['dataset']['data']

# temporary lists 
k = []  # to store date's to be used as unique keys
v = []  # to store remaining 'data' values corresponding to date

# loop through 'data' values separating 'Date' from 'Open', 'High', etc.
for val in vals:
    k.append(val[0])
    v.append(val[1:])

# use zip() to create more specific 'data' dictionary
# with date's as unique keys
data_dict = dict(zip(k, v))

***
### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [10]:
# temporary dictionary to store only {date:open prices}
open_prices = {}

# loop through data dictionary keys 
# to grab only date and open prices
# and create a new dictionary
for k in data_dict.keys():
    if data_dict[k][0] != None:  # exclude key:value pairs when open price = None
        open_prices[k]=data_dict[k][0]  # using index 0 for 'Open'

# call max() and min() functions on dictionary comprehensions
# that iterate through the key:value pairs and store in tuple
max_open_tup = max({(v,k) for (k,v) in open_prices.items()})
min_open_tup = min({(v,k) for (k,v) in open_prices.items()})

# output results
print("Highest opening price was {} on {}.".format(max_open_tup[0], max_open_tup[1]))
print("Lowest opening price was {} on {}.".format(min_open_tup[0], min_open_tup[1]))

Highest opening price was 53.11 on 2017-12-14.
Lowest opening price was 34.0 on 2017-01-24.


***
**Alternate approaches to Question 3**

In [11]:
# alternate approach could be to use max() and min() functions
# and include lambda function using the key= parameter
# which in this case will only get the value, not date
open_max = max(open_prices.keys(), key=(lambda k: open_prices[k]))
open_min = min(open_prices.keys(), key=(lambda k: open_prices[k]))
print(open_prices[open_max])
print(open_prices[open_min])

53.11
34.0


In [12]:
# another alternate approach could be to use a list and call 
# the max() and min() functions on the list but in this case
# I did not include date
open_prices = []

for v in data_dict.values():
    if v[0] != None :
        open_prices.append(v[0])

max_open = max(open_prices)
min_open = min(open_prices)

print("Highest opening price was {} and Lowest opening price was {}.".format(max_open, min_open))

Highest opening price was 53.11 and Lowest opening price was 34.0.


***
### 4. What was the largest change in any one day (based on High and Low price)?

In [13]:
# temporary dictionary to store only {date:diff} 
# where diff = high-low for given date
# not using 'Change'
price_diff = {}

# loop through data dictionary keys to 
# grab only date and high and low prices
# and create a new dictionary
for k in data_dict.keys():
    if (data_dict[k][1] != None) & (data_dict[k][2] != None):  # exclude key:value pairs when None
        # calculate price change
        price_diff[k]=data_dict[k][1]-data_dict[k][2]  # using indices 1 for 'High' and 2 for 'Low'

# call max() function on dictionary comprehension that 
# iterates through the key:value pairs and store in tuple
max_diff_tup = max({(v,k) for (k,v) in price_diff.items()})

# output results
print("Largest change (based on High/Low prices) was {} on {}.".format(max_diff_tup[0], max_diff_tup[1]))

Largest change (based on High/Low prices) was 2.8100000000000023 on 2017-05-11.


***
**Alternate approaches to Question 4**

In [14]:
# alternate approach could be to use a list and call 
# the max() function on the list but in this case
# I did not include date
price_diff = []

for v in data_dict.values():
    if (v[1] != None) & (v[2] != None):
        price_diff.append(v[1]-v[2])

max_diff = max(price_diff)

print("Largest change (based on High/Low prices) in any one day was {}.".format(max_diff))

Largest change (based on High/Low prices) in any one day was 2.8100000000000023.


***
### 5. What was the largest change between any two days (based on Closing Price)?

In [15]:
# temporary dictionary to store only {date:diff} 
# where diff = high-low for given date
close_prices = {}

# loop through data dictionary keys to 
# grab only date and high and low prices
# and create a new dictionary
for k in data_dict.keys():
    if data_dict[k][3] != None:  # exclude key:value pairs when None
        # calculate price change
        close_prices[k]=data_dict[k][3]  # using index 3 for 'Close'

# call max() function on dictionary comprehension that 
# iterates through the key:value pairs and store in tuple
max_close_tup = max({(v,k) for (k,v) in close_prices.items()})
min_close_tup = min({(v,k) for (k,v) in close_prices.items()})
max_diff = max_close_tup[0]-min_close_tup[0]

# output results
print("(Based on Closing prices)")
print("The larget change between any two days was between")
print("a MAX CLOSING PRICE of {} on {}".format(max_close_tup[0], max_close_tup[1]))
print("and MIN CLOSING PRICE of {} on {}".format(min_close_tup[0], min_close_tup[1]))
print("for a TOTAL CHANGE of {}.".format(max_diff))

(Based on Closing prices)
The larget change between any two days was between
a MAX CLOSING PRICE of 53.09 on 2017-12-13
and MIN CLOSING PRICE of 34.06 on 2017-01-23
for a TOTAL CHANGE of 19.03.


***
**Alternate approaches to Question 5**

In [16]:
# alternate approach could be to use a list and call 
# the max() and min() functions on the list but in this case
# I did not include any dates 
close_prices = []

for v in data_dict.values():
    if v[3] != None :
        close_prices.append(v[3])

max_diff = max(close_prices) - min(close_prices)

print("Largest change (based on Closing prices) between any two days was {}.".format(max_diff))

Largest change (based on Closing prices) between any two days was 19.03.


***
### 6. What was the average daily trading volume during this year?

In [17]:
# because only the average for the year is required, 
# I chose to use a list instead of dictionary

# list to store trading volume for year
volume = []

# loop through data dictionary keys to 
# grab only trading volumes for the year
# and store in list
for v in data_dict.values():
    if v[5] != None:
        volume.append(v[5])  # using index 5 for 'Traded Volume'

# calculate average and store as float
avg_volume = sum(volume) / len(volume)

# output result
print("Average daily trading volume during 2017 was {}.".format(avg_volume))

Average daily trading volume during 2017 was 89124.33725490196.


***
### (Optional) 
### 7. What was the median trading volume during this year.

In [18]:
# using volume list from question 6

# define function to calculate the median for given list
def get_median(my_list):
    """calculate the median for non-empty list of numbers;
       return the median value as float"""
    # get number of elements in list
    length = len(my_list)
    
    # check if empty list
    if length > 0:
        # if odd number of elements
        if length % 2 != 0:
            # sort and return average between two middle values
            return sorted(my_list)[int(length/2)]
        else:
            # sort and return middle value
            return sum(sorted(my_list)[int(length/2)-1 : int(length/2)+1])/2
    else:
        return None

# call get_median() function and store value
med_volume = get_median(volume)

# output result
print("Median trading volume during 2017 was {}.".format(med_volume))

Median trading volume during 2017 was 76286.0.
